# 第4回：AIの社会実装（Webアプリ開発）

## 学習目標
- 学習したモデルを**インタラクティブなWebアプリとして公開**する
- **Gradio**を用いて、誰でもブラウザからAI診断を体験できるようにする

## Gradioとは？
Gradioは、機械学習モデルを簡単にWebアプリ化できるPythonライブラリです。
- 数行のコードでUIを構築
- `share=True` で一時的な公開URLを発行可能（72時間有効）
- 友人や同僚に「自分が作ったAI」を共有できる

---
## Step 1: 環境セットアップ

In [ ]:
# 環境セットアップ
!pip install medmnist gradio -q
import sys, os

# リポジトリをクローン
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q

sys.path.insert(0, '/tmp/MedMNIST-Exercise')

# モジュールをインポート
sys.modules.pop('exercise_logic', None)
import exercise_logic

exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")

---
## Step 2: データセットの選択

第1回〜第3回で扱ったデータセットから1つを選んでください。

| データセット | 内容 | クラス数 |
|-------------|------|----------|
| pathmnist | 病理画像 | 9クラス |
| dermamnist | 皮膚病変 | 7クラス |
| chestmnist | 胸部X線 | 2クラス（正常/異常） |

In [ ]:
# ============================================
# 使用するデータセットを1つ選んでください
# 使いたい行のコメント(#)を外してください
# ============================================

data_flag = 'pathmnist'      # 第1回: 病理画像（9クラス分類）
# data_flag = 'dermamnist'   # 第2回: 皮膚病変（7クラス分類）
# data_flag = 'chestmnist'   # 第3回: 胸部X線（2値分類）

---
## Step 3: データ読み込みとモデル学習

選択したデータセットでモデルを学習します。

In [ ]:
# データの読み込み
# chestmnistの場合は2値分類に変換
if data_flag == 'chestmnist':
    (x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess(
        data_flag, 
        binary_classification=True
    )
    num_classes = 1
else:
    (x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess(data_flag)
    num_classes = len(info['label'])

print(f"Dataset: {data_flag}")
print(f"Training data: {x_train.shape}")
print(f"Labels: {info['label']}")

In [ ]:
# モデルの構築と学習
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=num_classes,
    model_type='simple'
)

print("モデルを学習中...（約2-3分）")
history = model.fit(
    x_train, y_train, 
    epochs=5, 
    validation_split=0.1, 
    batch_size=128,
    verbose=1
)
print("\n✓ モデル学習が完了しました。")

---
## Step 4: Gradio Webアプリの作成

学習したモデルをWebアプリとして公開します。

**機能:**
- 画像をアップロード
- AIが診断結果を予測
- 各クラスの確率を棒グラフで表示

In [ ]:
import gradio as gr
import numpy as np
from PIL import Image

# ラベル名を取得（英語表示用）
label_names = list(info['label'].values())

def predict(image):
    """
    アップロードされた画像を診断する関数
    """
    if image is None:
        return None
    
    # 画像の前処理
    img = Image.fromarray(image).convert('RGB').resize((28, 28))
    img_array = np.array(img).astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # 予測
    preds = model.predict(img_array, verbose=0)[0]
    
    # 2値分類の場合（sigmoid出力）
    if len(preds.shape) == 0 or len(label_names) == 2:
        prob = float(preds) if len(preds.shape) == 0 else float(preds[0])
        return {label_names[0]: 1 - prob, label_names[1]: prob}
    
    # 多クラス分類の場合（softmax出力）
    return {label_names[i]: float(preds[i]) for i in range(len(label_names))}

# Gradioインターフェースの構築
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(label="Upload Image"),
    outputs=gr.Label(num_top_classes=len(label_names), label="Prediction"),
    title="Medical Image Diagnosis AI",
    description=f"Dataset: {data_flag} | Upload an image to get AI diagnosis.",
    examples=None,
    flagging_mode="never"
)

print("✓ Gradioアプリの準備が完了しました。")
print("次のセルを実行してアプリを起動してください。")

---
## Step 5: サンプル画像のダウンロード

Webアプリで試すためのサンプル画像をダウンロードします。
テストデータから各クラス最低1枚を含む10枚の画像を選んでJPEGファイルとして保存します。
実行するたびに異なるサンプルが選ばれます。

In [ ]:
import numpy as np
from PIL import Image
import zipfile
import os

# サンプル画像を保存するディレクトリ
sample_dir = '/tmp/sample_images'
os.makedirs(sample_dir, exist_ok=True)

# 既存ファイルをクリア
for f in os.listdir(sample_dir):
    os.remove(os.path.join(sample_dir, f))

# ハイブリッドサンプリング: 各クラス最低1枚 + 残りはランダム
n_samples = 10
selected_indices = []

# 各クラスから1枚ずつ選択
unique_classes = np.unique(y_test)
for cls in unique_classes:
    cls_indices = np.where(y_test == cls)[0]
    selected_indices.append(np.random.choice(cls_indices))

# 残りをランダムに選択（重複なし）
remaining = n_samples - len(selected_indices)
if remaining > 0:
    available_indices = np.setdiff1d(np.arange(len(x_test)), selected_indices)
    additional = np.random.choice(available_indices, size=remaining, replace=False)
    selected_indices.extend(additional)

# 画像をJPEGとして保存
saved_files = []
for i, idx in enumerate(selected_indices):
    # 0-1のfloatを0-255のuint8に変換
    img_array = (x_test[idx] * 255).astype(np.uint8)
    img = Image.fromarray(img_array)
    
    # ファイル名にラベル情報を含める
    if data_flag == 'chestmnist':
        label = 'abnormal' if y_test[idx] == 1 else 'normal'
    else:
        label_idx = int(y_test[idx])
        label = info['label'][str(label_idx)]
    
    # ファイル名をシンプルに（特殊文字を除去、2桁連番）
    safe_label = label.replace(' ', '_').replace('-', '_')[:20]
    filename = f'sample_{i+1:02d}_{safe_label}.jpg'
    filepath = os.path.join(sample_dir, filename)
    
    img.save(filepath, 'JPEG')
    saved_files.append(filepath)
    print(f"Created: {filename} (Label: {label})")

# ZIPファイルにまとめる
zip_path = '/tmp/sample_images.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for filepath in saved_files:
        zipf.write(filepath, os.path.basename(filepath))

print(f"\n✓ サンプル画像を作成しました（{len(saved_files)}枚、各クラス最低1枚含む）")
print("次のセルを実行してダウンロードしてください。")

In [ ]:
# サンプル画像のダウンロード（Google Colab専用）
try:
    from google.colab import files
    files.download('/tmp/sample_images.zip')
    print("✓ ダウンロードが開始されました。")
    print("ダウンロードしたZIPファイルを解凍して、Webアプリで試してください。")
except ImportError:
    print("Google Colab以外の環境では、以下のパスからファイルを取得してください:")
    print(f"  {zip_path}")

---
## Step 6: アプリの起動

以下のセルを実行すると、Webアプリが起動します。

- **ローカルURL**: Colab内で直接使用可能
- **公開URL**: `share=True` により、72時間有効なURLが発行されます。このURLを共有すれば、誰でもあなたのAIを試すことができます。

**試し方:**
1. 上でダウンロードしたサンプル画像を解凍
2. Webアプリの「Upload Image」をクリック
3. サンプル画像を選択してアップロード
4. AIの予測結果を確認

In [ ]:
# アプリの起動（公開URLを発行）
demo.launch(share=True)

---
## 演習課題

### 課題1: 異なるデータセットで試す
Step 2でデータセットを変更し、再度全てのセルを実行してみましょう。

### 課題2: 考察
以下の点について考えてみましょう：

1. WebアプリとしてAIを公開することで、どのようなメリットがありますか？
2. 医療AIをWebアプリとして公開する場合、どのような注意点がありますか？
3. 実際の医療現場でAIを導入する際、どのような課題があると思いますか？

---
## まとめ

- **Gradio**: 機械学習モデルを簡単にWebアプリ化できるライブラリ
- **share=True**: 一時的な公開URLを発行し、誰でもアクセス可能に
- **AIの社会実装**: モデルを作るだけでなく、ユーザーが使える形で提供することが重要

### 全4回のまとめ

| 回 | テーマ | 学んだこと |
|----|--------|------------|
| 第1回 | CNN入門 | 畳み込みニューラルネットワークによる画像分類 |
| 第2回 | 転移学習 | 学習済みモデルの活用、クラス不均衡への対処 |
| 第3回 | Grad-CAM | AIの判断根拠の可視化、説明可能なAI |
| 第4回 | 社会実装 | Webアプリ化、AIの公開と共有 |